# Setup

In [ ]:
import os, json,re
from tqdm import tqdm 

## Bible books 

In [ ]:
corrected_books = {
    'Gen': 'Genesis',
    'Exo': 'Exodus',
    'Lev': 'Leviticus',
    'Num': 'Numbers',
    'Deu': 'Deuteronomy',
    'Jos': 'Joshua',
    'Jdg': 'Judges',
    'Rut': 'Ruth',
    '1Sa': '1 Samuel',
    '2Sa': '2 Samuel',
    '1Ki': '1 Kings',
    '2Ki': '2 Kings',
    '1Ch': '1 Chronicles',
    '2Ch': '2 Chronicles',
    'Ezr': 'Ezra',
    'Neh':'Nehemiah',
    'Est': 'Esther',
    'Job': 'Job',
    'Psa': 'Psalms',
    'Pro': 'Proverbs',
    'Ecc': 'Ecclesiastes',
    'Sng': 'Canticles',
    'Isa': 'Isaiah',
    'Jer': 'Jeremiah',
    'Lam': 'Lamentations',
    'Ezk': 'Ezekiel',
    'Dan': 'Daniel',
    'Hos': 'Hosea',
    'Jol': 'Joel',
    'Amo': 'Amos',
    'Oba': 'Obadiah',
    'Jon': 'Jonah',
    'Mic': 'Micah',
    'Nam': 'Nahum',
    'Hab': 'Habakkuk',
    'Zep': 'Zephaniah',
    'Hag': 'Haggai',
    'Zec': 'Zechariah',
    'Mal': 'Malachi',
    'Mat': 'Matthew',
    'Mrk': 'Mark',
    'Luk': 'Luke',
    'Jhn': 'John',
    'Act': 'Acts',
    'Rom': 'Romans',
    '1Co': '1 Corinthians',
    '2Co': '2 Corinthians',
    'Gal': 'Galatians',
    'Eph': 'Ephesians',
    'Php': 'Philippians',
    'Col': 'Colossians',
    '1Th': '1 Thessalonians',
    '2Th': '2 Thessalonians',
    '1Ti': '1 Timothy',
    '2Ti': '2 Timothy',
    'Tit': 'Titus',
    'Phm': 'Philemon',
    'Heb': 'Hebrews',
    'Jas': 'James',
    '1Pe': '1 Peter',
    '2Pe': '2 Peter',
    '1Jn': '1 John',
    '2Jn': '2 John',
    '3Jn': '3 John',
    'Jud': 'Jude',
    'Rev': 'Revelation',
    'Sus': "Susanna",
    'Bel':"Bel and the Dragon",
    'Bar':'Baruch',
    'Sir':'Ecclesiasticus',
    'Wis':'Wisdom',
    '1Ma':'1 Maccabees',
    '2Ma':'2 Maccabees',
    '1Es':"1 Esdras",
    '2Es':'2 Esdras',
    '3Es': '3 Esdras',
    '4Es': '4 Esdras',
    'Tob':'Tobit',
    'Jdt':'Judith',
    '4Ma':'4 Maccabees',
    'Man':'Manasseh',
    'S3Y':"Song of the Three Young Men",
    'Oda':'Obadiah'
}

# Versification

## Get verse mappings

In [ ]:
def convert_test(expression):
    # Replace abbreviations with full names in the start and end references
    def replace_match(match):
        abbrev = match.group(1)
        chapter = match.group(2)
        verse = match.group(3)
        full_book = corrected_books.get(abbrev, abbrev)
        return f"{full_book} {chapter}.{verse}"

    # Split the expression by logical operators (including '>', '<', '=', etc.)
    expression = re.sub(r'([\d\w]+)\.(\d+):(\d+(\.\d+)?)', replace_match, expression)

    # Identify the logical operator and split accordingly
    if '=' in expression:
        parts = expression.split('=')
        operation = 'LAST' if 'Last' in parts[1] else 'E' if 'Exist' in parts[1] else 'NE'
        start = parts[0].strip()
        end = None
    elif '>' in expression:
        parts = expression.split('>')
        start = parts[0].strip()
        end = parts[1].strip()
        operation = 'GT'
    elif '<' in expression:
        parts = expression.split('<')
        start = parts[0].strip()
        end = parts[1].strip()
        operation = 'LT'
    else:
        # If no operator, assume it's a single reference with an operator
        start = expression.strip()
        end = None
        operation = 'E'

    return (start, end, operation)

In [508]:
with open(f"../../Bibles/TVTMS - Translators Versification Traditions with Methodology for Standardisation for Eng+Heb+Lat+Grk+Others - STEPBible.org CC BY.txt") as file: 
    data = file.readlines()

in_section = False 
tradition_tests = {} # AKJV to Latin and Hebrew  
mapping = {}
test_key = None 
def convert_name(item): 
    if "Ps2" in item or 'Esg' in item or "LJe" in item or "Absent" in item: return None 
    if "." not in item: return None 
    book, nums = item.split(".")[:2]
    book = corrected_books[book]
    nums = re.sub(":",".",nums)
    return f"{book} {nums}"

for idx, line in enumerate(data): 
    line = line.strip()
    if not line:
        continue
    if re.search(r'\$Rom.14:22-23', line): continue 
    if re.search(r"\$[\d\w]+\.\d+\:",line):
        if "English KJV" not in line: continue 
        line = line.split("\t")
        key = line[0]
        if "-" in key: # across chapter division 
            if "--" in key: 
                start, end = key.strip("$#").split("--")
            else: 
                start, end = key.strip("$#").split("-")
            # print(start,end)
            book,start = start.split(".")[:2]
            if book not in corrected_books: 
                in_section = False 
                continue 
            in_section = True 
            book = corrected_books[book]
            start = re.sub(":",".",start)            
            end = re.sub(":",".", end)
            test_key = ";".join([book, start, end])
            tradition_tests[test_key] = {k:[] for k in line[1:]} 
        else: 
            key = key.strip("$#$")
            book,start = key.split(".")[:2]
            if book not in corrected_books: 
                in_section = False 
                continue 
            in_section = True 
            book = corrected_books[book]
            start = re.sub(":",".",start)
            test_key = ";".join([book, start, start])
            tradition_tests[test_key] = {k:[] for k in line[1:]} 
    if "TEST: If..." in line and in_section: 
        line = line.split("\t")[1:]
        if len(line) != len(tradition_tests[test_key]): continue 
        for idx, key in enumerate(tradition_tests[test_key]):
            test = line[idx]
            tradition_tests[test_key][key].append(convert_test(line[idx]))
    elif ("To" in line or "Verse" in line) and in_section: 
        line = line.split("\t")[1:]
        if len(line) != len(tradition_tests[test_key]): continue 
        verse_key = None 
        for idx, key in enumerate(tradition_tests[test_key]): 
            if key == "English KJV": 
                verse_key = convert_name(line[idx])
        if verse_key is None: continue
        if "Title" in verse_key: continue  
        for idx, key in enumerate(tradition_tests[test_key]): 
            if verse_key not in mapping: 
                mapping[verse_key] = {}
            if "Absent" in line[idx] or 'NoVerse' in line[idx]: 
                mapping[verse_key][key] = None
            else: 
                mapping[verse_key][key] = convert_name(line[idx])

In [510]:
tradition_tests['Exodus;39.1;39.43']

{'English KJV': [('Exodus 39.43', None, 'LAST'),
  ('Exodus 36.38', None, 'LAST'),
  ('Exodus 39.19', 'Exodus 39.21', 'LT')],
 'Hebrew': [('Exodus 39.43', None, 'LAST'),
  ('Exodus 36.38', None, 'LAST'),
  ('Exodus 39.19', 'Exodus 39.21', 'LT')],
 'Latin': [('Exodus 39.43', None, 'LAST'),
  ('Exodus 36.38', None, 'LAST'),
  ('Exodus 39.19', 'Exodus 39.21', 'GT')],
 'Greek*': [('Exodus 39.23', None, 'LAST'),
  ('Exodus 36.38', None, 'LAST'),
  ('Exodus 39.19', 'Exodus 39.21', 'LT')],
 'Greek2 (eg NETS)*': [('Exodus 39.23', None, 'LAST'),
  ('Exodus 36.40', None, 'LAST'),
  ('Exodus 39.19', 'Exodus 39.21', 'LT')]}

In [493]:
def expand_references(reference_list):
    book = reference_list[0]
    start_ref = reference_list[1]
    end_ref = reference_list[2]

    # Split start_ref and end_ref into chapter and verse
    start_chapter, start_verse = map(int, start_ref.split('.'))
    if "." not in end_ref: 
        end_chapter, end_verse = start_chapter, int(end_ref) 
    else: 
        end_chapter, end_verse = map(int, end_ref.split('.'))

    result = []

    # Iterate through chapters and verses
    for chapter in range(start_chapter, end_chapter + 1):
        # Set start and end verses depending on the chapter
        start_v = start_verse if chapter == start_chapter else 1
        end_v = end_verse if chapter == end_chapter else 151  # Assuming 151 is a safe upper bound

        for verse in range(start_v, end_v + 1):
            full_ref = f'{book} {chapter}.{verse}'
            result.append(full_ref)
    return result


In [494]:
final_mapping = {}
for v_id, mlist in mapping.items(): 
    if "-" in v_id: 
        book = " ".join(v_id.split()[:-1])
        cv = v_id.split()[-1]
        cv1, cv2 = v_id.split()[-1].split("-")
        refs = expand_references([book,cv1,cv2])
        for ref in refs: 
            final_mapping[ref] = {}
        for trad, v_id2 in mlist.items():
            if v_id2 is None: 
                for idx, ref in enumerate(refs): 
                    final_mapping[ref][trad] = None 
            else: 
                book = " ".join(v_id2.split()[:-1])
                cv = v_id2.split()[-1].split("-")
                if len(cv) == 2: 
                    cv1, cv2 = cv
                    refs2 = expand_references([book,cv1,cv2])
                    for idx, ref in enumerate(refs): 
                        if len(refs2) > idx: 
                            final_mapping[ref][trad] = refs2[idx]
                else: 
                    for ref in refs: 
                        final_mapping[ref][trad] = v_id2
    else: 
        final_mapping[v_id] = mlist
mapping = final_mapping 

In [495]:
with open("../../Bibles/versification.json",'w+') as file: 
    json.dump([tradition_tests, mapping],file)

## Get Parallel Verses

In [496]:
bible = {}
import pandas as pd 
b_versions = ['AKJV','Geneva', 'ODRV','Douay-Rheims', 'Tyndale', 'Wycliffe','Vulgate']
for bname in b_versions:
    data = pd.read_csv(f"../../Bibles/{bname}.csv",header=None)
    data = data.to_dict(orient="records")
    for entry in tqdm(data):
        key = entry[0]
        v_id = key.split(" (")[0]
        text = entry[6]
        if re.search("Douay-Rheims",key):
            if re.sub("Douay-Rheims","ODRV",key) in bible: continue
        if len(text.split(" ")) < 200:
            bible[key] = f"{v_id} {text}"

        parts = re.split(r'(?<=[\.\?]) (?=[A-Z])|(?<=[\!\:\;])', text)
        parts = [re.sub(r'\s+', ' ', p).strip() for p in parts if len(p.strip(" ")) > 0]
        if (len(parts[0].split(" ")) <= 5 or len(parts[-1].split(" ")) <= 5 or re.search(r"\&\w+\;",parts[0])): 
            for pidx, p in enumerate(parts): continue
        elif len(parts) > 1:  
            for pidx, p in enumerate(parts):
              p_id = f"{key} - {pidx}"
              if len(p) == 0: continue
              if re.search(r"\&\w+\;",p) or len(p.split(" ")) <= 5: continue
              bible[p_id] = f"Part {pidx+1} of {v_id}: {p}"
bible_verses = list(bible.values())
bible_ids = list(bible.keys())
id_to_idx = {v_id:idx for idx, v_id in enumerate(bible_ids)}
len(bible_verses)

100%|██████████| 35809/35809 [00:01<00:00, 28843.56it/s]


276627

In [581]:
def fix_name(v_id):
  if "1 Kings" in v_id: v_id = "3 Kings" + v_id.split("1 Kings")[-1]
  elif "2 Kings" in v_id: v_id = "4 Kings" + v_id.split("2 Kings")[-1]
  elif "1 Samuel" in v_id: v_id = "1 Kings" + v_id.split("1 Samuel")[-1]
  elif "2 Samuel" in v_id: v_id = "2 Kings" + v_id.split("2 Samuel")[-1]
  elif re.search(r"^\d+ Chronicles",v_id): v_id = re.sub(r"Chronicles","Paralipomenon",v_id)
  return v_id

def expand_test_id(reference_list, version):
    book = reference_list[0]
    if version in ['Douay-Rheims','Vulgate','ODRV']: 
        book = fix_name(book)
    start_ref = reference_list[1].strip("abc")
    end_ref = reference_list[2].strip('abc')
    start_chapter, start_verse = map(int, start_ref.split('.'))
    if "." not in end_ref: 
        end_chapter, end_verse = start_chapter, int(end_ref) 
    else: 
        end_chapter, end_verse = map(int, end_ref.split('.'))

    result = []

    for chapter in range(start_chapter, end_chapter + 1):
        start_v = start_verse if chapter == start_chapter else 1
        end_v = end_verse if chapter == end_chapter else 151

        for verse in range(start_v, end_v + 1):
            full_ref = f'{book} {chapter}.{verse} ({version})'
            if full_ref in bible: 
                result.append(full_ref)
    return result

def normalize_cv(cv):
    cv = cv.split("*")[0]
    parts = cv.split(".")
    return ".".join(parts[:2])

def E(book, cv, version):
    ref = f"{book} {normalize_cv(cv)} ({version})"
    return ref in bible

def NE(book, cv, version):
    ref = f"{book} {normalize_cv(cv)} ({version})"
    return ref not in bible

def LT(book1, cv1, book2, cv2, version):
    ref1 = f"{book1} {normalize_cv(cv1)} ({version})"
    ref2 = f"{book2} {normalize_cv(cv2)} ({version})"
    return ref1 in bible and ref2 in bible and len(bible[ref1].split()) < len(bible[ref2].split())

def GT(book1, cv1, book2, cv2,  version):
    ref1 = f"{book1} {normalize_cv(cv1)} ({version})"
    ref2 = f"{book2} {normalize_cv(cv2)} ({version})"
    return ref1 in bible and ref2 in bible and len(bible[ref1].split()) > len(bible[ref2].split())

def LAST(book, cv,  version):
    chapter, verse = map(int, normalize_cv(cv).split("."))
    v = 1
    while f"{book} {chapter}.{v} ({version})" in bible:
        v += 1
    return int(verse) == v - 1

In [572]:
def verse_comparison(bcv_list,cv1): 
    if "." not in bcv_list[-1]: 
        v = bcv_list[-1]
        c = bcv_list[-2].split(".")[0]
        cv = f"{c}.{v}"
    else: 
        cv = normalize_cv(bcv_list[-1])
        c, v = cv.split(".")

    cv1 = normalize_cv(cv1)
    c1, v1 = cv1.split(".")
    if "-" in v1:
        v1 = v1.split("-")[-1] 

    v = v.strip('abc')
    v1 = v1.strip("abc")
    c,v,c1,v1 = int(c), int(v), int(c1), int(v1)
    if c>=c1 and v>=v1: 
        return cv
    else: 
        return cv1 

In [578]:
PV = {}

for ref, tests in tradition_tests.items(): 
    if "Title" in ref: continue
    ref = ref.split(";")
    
    for trad, tlist in tests.items(): 
        if "English KJV" not in trad: 
            continue
        for test in tlist: 
            if test is None: continue 
            if test[1] is None: 
                if 'Title' in test[0]: continue 
                ref[-1] = verse_comparison(ref,test[0].split()[-1])
            else: 
                ref[-1] =  verse_comparison(ref,test[1].split()[-1])
    ref[-1] = normalize_cv(ref[-1])
    ref[-2] = normalize_cv(ref[-2])   
     
    if "-" in ref[-1]: 
        c,v = re.findall(r'(\d+).\d+-(\d+)',ref[-1])[0]
        ref[-1] = f"{c}.{v}"
    all_refs = expand_test_id(ref,'AKJV')
    for r in all_refs: 
        PV[r] = []

    for ver in b_versions: 
        if ver == 'AKJV': continue 
        for trad, tlist in tests.items():
            passed = True  
            for test in tlist:  
                if test is None: 
                    continue 
                cv = test[0].split()[-1]
                book = " ".join(test[0].split()[:-1])
                if ver in ['Douay-Rheims','ODRV','Vulgate']: 
                    book = fix_name(book)
                if test[1] is not None: 
                    cv2 = test[1].split()[-1]
                    book2 = " ".join(test[1].split()[:-1])
                    if ver in ['Douay-Rheims','ODRV','Vulgate']: 
                        book2 = fix_name(book2)
                if test[-1] == 'LAST':
                    result = LAST(book, cv, ver)
                elif test[-1] == "E": 
                    result = E(book, cv, ver)
                elif test[-1] == "NE": 
                    result = NE(book,cv,ver)
                elif test[-1] == "LT": 
                    result = LT(book, cv, book2, cv2, ver)
                if not result: 
                    passed = False 
            
            if passed: 
                for r in all_refs:
                    new_key = r.split(" (")[0]
                    if new_key not in mapping: 
                        continue 
                    new_id = mapping[new_key][trad]
                    if new_id is None: 
                        continue 

                    if "-" in new_id: 
                        b = " ".join(new_id.split()[:-1])
                        cv1,cv2 = new_id.split()[-1].split("-")
                        new_ids = expand_test_id([b,cv1,cv2],ver) 
                    else: 
                        if ver in ['Douay-Rheims','ODRV','Vulgate']: 
                            new_id = fix_name(new_id)
                        new_ids = [new_id + f" ({ver})"]
                    for new_id in new_ids: 
                        if new_id in bible:  
                            PV[r].append(new_id)
                break

In [579]:
print(len(PV))
AKJV = [v_id for v_id in bible if "AKJV" in v_id and "-" not in v_id]
for v_id in AKJV: 
    if v_id in PV: continue 
    for ver in b_versions: 
        if ver == "AKJV": continue 
        key = v_id.split(" (")[0]
        if f"{key} ({ver})" in bible: 
            if v_id not in PV: PV[v_id] = []
            PV[v_id].append(f"{key} ({ver})")
print(len(PV))

8548
35440


In [580]:
with open(f"../../Bibles/PARALLEL_VERSES.json",'w+') as file: 
    json.dump(PV, file)

In [501]:
PV['Daniel 4.12 (AKJV)']

['Daniel 4.9 (Geneva)', 'Daniel 4.9 (ODRV)', 'Daniel 4.9 (Vulgate)']

In [502]:
PV['1 Kings 22.53 (AKJV)']

['1 Kings 22.53 (Geneva)',
 '3 Kings 22.54 (Douay-Rheims)',
 '3 Kings 22.54 (Vulgate)']

In [575]:
PV['Exodus 39.30 (AKJV)']

['Exodus 39.30 (Geneva)',
 'Exodus 39.29 (ODRV)',
 'Exodus 39.29 (Wycliffe)',
 'Exodus 39.29 (Vulgate)']

In [507]:
mapping['Exodus 39.30']

{'English KJV': 'Exodus 39.30',
 'Hebrew': 'Exodus 39.30',
 'Latin': 'Exodus 39.29',
 'Greek*': 'Exodus 36.37',
 'Greek2 (eg NETS)*': 'Exodus 36.38-39'}

# Proper Nouns 

In [ ]:
# Biblical entities 
with open(f"../assets/Bibles/TIPNR - Translators Individualised Proper Names with all References - STEPBible.org CC BY.txt") as file: 
    data = file.readlines()
in_entities_section = False
e_to_v = {}
for idx, line in enumerate(data): 
    line = line.strip()
    if not line:
        continue
    
    if line.startswith('$========== '):
        in_entities_section = True
        continue  
    
    if idx < 112: continue
    if idx > 14394: continue
    if in_entities_section:
        if line[0] != "–": continue 
        if not re.search("@",line): continue 
        name = re.findall("^(.*?)@",line)[0]
        refs = line.split("reference=")[-1].split("\t")[-1]
        name = name.split("\t")[-1].split("|")
        KJV_name = re.findall(r"KJV\s*=\s*(.*?)[);,]",line)
        if len(KJV_name) > 0: 
            KJV_name = KJV_name[0]
            name.append(KJV_name)
        for n in name: 
            n = re.sub("_"," ",n)
            if n not in e_to_v: e_to_v[n] = []
            ref_list = [re.findall("([\d\w]+)\.(\d+\.\d+)",r) for r in refs.split("; ")]
            ref_list = [f"{corrected_books[r[0][0]]} {r[0][1]}" for r in ref_list if len(r) > 0 if r[0][0] != 'Etc']
            e_to_v[n].extend(sorted(list(set(ref_list))))
    else:
        continue

In [ ]:
with open(f"../assets/Bibles/proper_nouns.json",'w+') as file: 
    json.dump(e_to_v, file)

# Citations 

In [ ]:
import pandas as pd
from tqdm import tqdm  
def clean_text(s): 
    s = re.sub(r"\<\/i\>|\<NOTE\>|NONLATINALPHABET|\<i\>|\d+\^PAGE[S]*\^MISSING","",s)
    s = re.sub(r"\s+"," ",s)
    s = s.strip(" ")
    return s 
all_c = {}
c_repo = "../../CITATIONS"
c_files = sorted(k for k in os.listdir(c_repo) if ".csv" in k)
for file in tqdm(c_files): 
    era, prefix = file.split("_citations.csv")[0].split("_")
    # print(era,prefix)
    data = pd.read_csv(f"{c_repo}/{file}",header=None).to_dict(orient='records')
    data = {(d[0],d[1],d[2]):d for d in data}
    text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{prefix}_body.csv",header=None).to_dict(orient='records')
    text = {(d[0],d[1],"In-Text"):clean_text(d[6]) for d in text}
    for key, t in text.items(): 
        if key in data: 
            data[key]['text'] = t 
    if f"{prefix}_margin.csv" in os.listdir(f"../../SERMONS_APP/db/data/{era}"): 
        text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{prefix}_margin.csv",header=None).to_dict(orient='records')
        text = {(d[0],d[1],"Note " + str(d[2])):clean_text(d[3]) for d in text}
        for key, t in text.items(): 
            if key in data: 
                data[key]['text'] = t
    for d in data.values(): 
        all_c[d[6]] = d[4] 
    # data = pd.DataFrame(list(data.values()))
    # data.to_csv(f"{c_repo}/{file}",index=False)

In [ ]:
list(all_c.items())[1:10]

In [ ]:
import sys 
sys.path.append('../')
from lib.EEPS_helper import isNumeral
from collections import Counter

In [ ]:
formats = {}

def space_punctuation(text):
    text = re.sub(r'([\.\,\:\;\!\?\(\)\-\&])(?=\w)', r'\1 ', text)
    text = re.sub(r'(?<=\w)([\.\,\:\;\!\?\(\)\-\&])', r' \1', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

for orig in tqdm(all_c): 
    if orig == '6': continue 
    text = space_punctuation(orig)
    tokens = text.split(" ")
    format = []
    for t in tokens: 
        if isNumeral(t): 
            format.append('N')
        elif re.search(r"[\.\,\:\;\!\?\(\)\-\&]",t):  
            format.append(t)
        else: 
            format.append('W')
    format = " ".join(format)
    if format not in formats: formats[format] = 0
    formats[format] += 1       

In [ ]:
Counter(formats).most_common(n=10)

In [ ]:
all_books = {}
for orig, std in all_c.items(): 
    if orig == '6' and std == '4': continue 
    words = re.findall(r"([A-Za-z]+)",orig)
    book = re.findall(r"[A-Za-z]+",std)[0].lower()
    if book not in all_books: 
        all_books[book] = {}
    for w in words: 
        if isNumeral(w): 
            continue 
        w = w.lower()
        if w in ['of','cap','chap','verse','vers','chapter','ch']:
            continue 
        if w not in all_books[book]: 
            all_books[book][w] = 0 
        all_books[book][w] += 1 

common_abbrev = {}
for book, abbrevs in all_books.items(): 
    if book == "children": continue 
    top = Counter(abbrevs).most_common(n=1)[0]
    common_abbrev[book] = top[0] 

with open(f"../assets/Bibles/top_abbrev.json",'w+') as file: 
    json.dump(common_abbrev, file)
common_abbrev 